# Notebook 2: MCP Primitives Deep Dive

## Best Practices for Agentic AI using MCP

---

**Series:** MCP Server Best Practices - Agentic AI Workflows  
**Notebook:** 2 of 13  
**Level:** Beginner to Intermediate  
**Duration:** ~60 minutes

---

## Learning Objectives

By the end of this notebook, you will:

1. Master the design of MCP **Tools** with proper JSON Schemas
2. Understand **Resource** types, URIs, and subscriptions
3. Create effective **Prompt** templates for agent workflows
4. Learn the MCP methods for each primitive (`list`, `call`, `read`, `get`)
5. Understand control models (Model, Application, User-controlled)
6. Build practical examples of each primitive

## 1. Recap: The Three Pillars of MCP

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    MCP PRIMITIVES OVERVIEW                               │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   TOOLS                 RESOURCES               PROMPTS                 │
│   ═════                 ═════════               ═══════                 │
│                                                                          │
│   🔧 Execute            👁️ Read Data            📋 Guide Behavior       │
│                                                                          │
│   • Actions             • Files                 • Templates             │
│   • Side effects        • Database rows         • Best practices        │
│   • API calls           • API responses         • Workflows             │
│   • State changes       • Configurations        • Multi-step guides     │
│                                                                          │
│   Model decides         App/User selects        User invokes            │
│   when to call          what to fetch           explicitly              │
│                                                                          │
│   tools/list            resources/list          prompts/list            │
│   tools/call            resources/read          prompts/get             │
│                         resources/subscribe                             │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Setup for this notebook
import json
from typing import Dict, List, Any, Optional
from dataclasses import dataclass, field
from enum import Enum
from datetime import datetime

# Pretty print helper
def pprint(obj: dict, title: str = None):
    if title:
        print(f"\n{'='*60}")
        print(f" {title}")
        print(f"{'='*60}")
    print(json.dumps(obj, indent=2, default=str))

## 2. Tools: Actions for the AI Agent

### 2.1 What Makes a Good Tool?

Tools are the **hands** of your AI agent. They must be:

| Characteristic | Description | Example |
|---------------|-------------|----------|
| **Single Purpose** | One clear action | `sendEmail` not `manageEmails` |
| **Well-Described** | Clear name & docs | "Search flights between airports" |
| **Strongly Typed** | JSON Schema inputs | `{"origin": "string", "date": "date"}` |
| **Predictable** | Same input = same behavior | Deterministic operations |
| **Safe** | Fail gracefully | Return errors, don't crash |

### 2.2 Tool Definition Structure

Every MCP tool has this structure:

```json
{
  "name": "toolName",
  "description": "What the tool does",
  "inputSchema": {
    "type": "object",
    "properties": { ... },
    "required": [ ... ]
  }
}
```

In [ ]:
# Example: Complete tool definitions

# Tool 1: Simple tool with basic types
calculator_tool = {
    "name": "calculate",
    "description": "Perform basic arithmetic operations (add, subtract, multiply, divide)",
    "inputSchema": {
        "type": "object",
        "properties": {
            "operation": {
                "type": "string",
                "enum": ["add", "subtract", "multiply", "divide"],
                "description": "The arithmetic operation to perform"
            },
            "a": {
                "type": "number",
                "description": "First operand"
            },
            "b": {
                "type": "number",
                "description": "Second operand"
            }
        },
        "required": ["operation", "a", "b"]
    }
}

pprint(calculator_tool, "TOOL 1: Calculator")

In [ ]:
# Tool 2: Complex tool with nested objects
flight_search_tool = {
    "name": "searchFlights",
    "description": "Search for available flights between airports with optional filters",
    "inputSchema": {
        "type": "object",
        "properties": {
            "origin": {
                "type": "string",
                "pattern": "^[A-Z]{3}$",
                "description": "Origin airport IATA code (e.g., 'JFK')"
            },
            "destination": {
                "type": "string",
                "pattern": "^[A-Z]{3}$",
                "description": "Destination airport IATA code (e.g., 'LAX')"
            },
            "departureDate": {
                "type": "string",
                "format": "date",
                "description": "Departure date (YYYY-MM-DD)"
            },
            "returnDate": {
                "type": "string",
                "format": "date",
                "description": "Optional return date for round trips"
            },
            "passengers": {
                "type": "object",
                "properties": {
                    "adults": {"type": "integer", "minimum": 1, "maximum": 9},
                    "children": {"type": "integer", "minimum": 0, "maximum": 8},
                    "infants": {"type": "integer", "minimum": 0, "maximum": 4}
                },
                "required": ["adults"]
            },
            "cabinClass": {
                "type": "string",
                "enum": ["economy", "premium_economy", "business", "first"],
                "default": "economy"
            },
            "maxPrice": {
                "type": "number",
                "minimum": 0,
                "description": "Maximum price per person in USD"
            }
        },
        "required": ["origin", "destination", "departureDate", "passengers"]
    }
}

pprint(flight_search_tool, "TOOL 2: Flight Search (Complex)")

In [ ]:
# Tool 3: Tool with array parameters
bulk_email_tool = {
    "name": "sendBulkEmail",
    "description": "Send personalized emails to multiple recipients",
    "inputSchema": {
        "type": "object",
        "properties": {
            "recipients": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "email": {"type": "string", "format": "email"},
                        "name": {"type": "string"},
                        "variables": {
                            "type": "object",
                            "additionalProperties": {"type": "string"}
                        }
                    },
                    "required": ["email"]
                },
                "minItems": 1,
                "maxItems": 100
            },
            "subject": {
                "type": "string",
                "maxLength": 200
            },
            "bodyTemplate": {
                "type": "string",
                "description": "Email body with {{variable}} placeholders"
            },
            "attachments": {
                "type": "array",
                "items": {
                    "type": "string",
                    "format": "uri",
                    "description": "URL to attachment file"
                }
            }
        },
        "required": ["recipients", "subject", "bodyTemplate"]
    }
}

pprint(bulk_email_tool, "TOOL 3: Bulk Email (Arrays)")

### 2.3 Tool Call and Response

The JSON-RPC flow for tool usage:

In [ ]:
# tools/list - Discover available tools
tools_list_request = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "tools/list",
    "params": {}
}

tools_list_response = {
    "jsonrpc": "2.0",
    "id": 1,
    "result": {
        "tools": [
            calculator_tool,
            flight_search_tool
        ]
    }
}

pprint(tools_list_request, "REQUEST: tools/list")
print("\n... Server responds with available tools ...")

In [ ]:
# tools/call - Execute a tool
tools_call_request = {
    "jsonrpc": "2.0",
    "id": 2,
    "method": "tools/call",
    "params": {
        "name": "searchFlights",
        "arguments": {
            "origin": "JFK",
            "destination": "LAX",
            "departureDate": "2025-02-15",
            "passengers": {
                "adults": 2,
                "children": 1
            },
            "cabinClass": "business"
        }
    }
}

tools_call_response = {
    "jsonrpc": "2.0",
    "id": 2,
    "result": {
        "content": [
            {
                "type": "text",
                "text": json.dumps({
                    "flights": [
                        {
                            "airline": "Delta",
                            "flightNumber": "DL123",
                            "departure": "08:00",
                            "arrival": "11:30",
                            "price": 850
                        },
                        {
                            "airline": "United",
                            "flightNumber": "UA456",
                            "departure": "14:00",
                            "arrival": "17:30",
                            "price": 780
                        }
                    ],
                    "totalResults": 2
                })
            }
        ],
        "isError": False
    }
}

pprint(tools_call_request, "REQUEST: tools/call")
pprint(tools_call_response, "RESPONSE: Tool Result")

### 2.4 Tool Design Best Practices

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    TOOL DESIGN BEST PRACTICES                            │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   ✅ DO                                ❌ DON'T                          │
│   ════                                 ══════                            │
│                                                                          │
│   • Use descriptive names             • Use cryptic abbreviations       │
│     "createCalendarEvent"               "cce"                           │
│                                                                          │
│   • Add detailed descriptions         • Leave descriptions empty        │
│     "Book a hotel room for             "Books hotel"                    │
│      specified dates and guests"                                        │
│                                                                          │
│   • Use JSON Schema validation        • Accept any input                │
│     {"pattern": "^[A-Z]{3}$"}          {"type": "string"}               │
│                                                                          │
│   • Provide sensible defaults         • Require all parameters          │
│     {"default": "economy"}             (makes tool harder to use)       │
│                                                                          │
│   • Return structured errors          • Throw exceptions                │
│     {"isError": true, "error":...}     (crashes the connection)         │
│                                                                          │
│   • Keep tools focused                • Create "god tools"              │
│     One tool = One action               "manageEverything"              │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

## 3. Resources: Data for the AI Agent

### 3.1 Understanding Resources

Resources are **passive data sources** that provide context to the model:

| Type | Description | Example URI |
|------|-------------|-------------|
| **Direct** | Fixed URI to specific data | `file:///reports/q4.pdf` |
| **Template** | Parameterized URI pattern | `weather://forecast/{city}` |

Resources are **read-only** - they don't modify state.

In [ ]:
# Direct Resources - Fixed URIs
direct_resources = [
    {
        "uri": "file:///config/app-settings.json",
        "name": "Application Settings",
        "description": "Current application configuration",
        "mimeType": "application/json"
    },
    {
        "uri": "file:///logs/system.log",
        "name": "System Logs",
        "description": "Recent system log entries",
        "mimeType": "text/plain"
    },
    {
        "uri": "db://users/current",
        "name": "Current User Profile",
        "description": "Profile data for the authenticated user",
        "mimeType": "application/json"
    },
    {
        "uri": "calendar://events/today",
        "name": "Today's Events",
        "description": "All calendar events for today",
        "mimeType": "application/json"
    }
]

pprint({"resources": direct_resources}, "DIRECT RESOURCES")

In [ ]:
# Resource Templates - Parameterized URIs
resource_templates = [
    {
        "uriTemplate": "weather://forecast/{city}/{date}",
        "name": "Weather Forecast",
        "description": "Get weather forecast for a specific city and date",
        "mimeType": "application/json"
    },
    {
        "uriTemplate": "db://users/{userId}/profile",
        "name": "User Profile",
        "description": "Get profile data for a specific user",
        "mimeType": "application/json"
    },
    {
        "uriTemplate": "file:///{path}",
        "name": "File Contents",
        "description": "Read contents of any file by path",
        "mimeType": "application/octet-stream"
    },
    {
        "uriTemplate": "github://repos/{owner}/{repo}/issues/{issueNumber}",
        "name": "GitHub Issue",
        "description": "Get details of a specific GitHub issue",
        "mimeType": "application/json"
    }
]

pprint({"resourceTemplates": resource_templates}, "RESOURCE TEMPLATES")

### 3.2 Resource Methods

MCP provides three methods for resources:

In [ ]:
# resources/list - Discover available resources
resources_list_request = {
    "jsonrpc": "2.0",
    "id": 10,
    "method": "resources/list",
    "params": {}
}

resources_list_response = {
    "jsonrpc": "2.0",
    "id": 10,
    "result": {
        "resources": direct_resources
    }
}

pprint(resources_list_request, "REQUEST: resources/list")

In [ ]:
# resources/templates/list - Discover resource templates
templates_list_request = {
    "jsonrpc": "2.0",
    "id": 11,
    "method": "resources/templates/list",
    "params": {}
}

templates_list_response = {
    "jsonrpc": "2.0",
    "id": 11,
    "result": {
        "resourceTemplates": resource_templates
    }
}

pprint(templates_list_request, "REQUEST: resources/templates/list")

In [ ]:
# resources/read - Fetch resource contents
resources_read_request = {
    "jsonrpc": "2.0",
    "id": 12,
    "method": "resources/read",
    "params": {
        "uri": "weather://forecast/NYC/2025-01-20"
    }
}

resources_read_response = {
    "jsonrpc": "2.0",
    "id": 12,
    "result": {
        "contents": [
            {
                "uri": "weather://forecast/NYC/2025-01-20",
                "mimeType": "application/json",
                "text": json.dumps({
                    "city": "New York City",
                    "date": "2025-01-20",
                    "high": 45,
                    "low": 32,
                    "conditions": "Partly Cloudy",
                    "precipitation": "10%"
                })
            }
        ]
    }
}

pprint(resources_read_request, "REQUEST: resources/read")
pprint(resources_read_response, "RESPONSE: Resource Contents")

### 3.3 Resource Subscriptions

Resources can be **subscribable** - the server pushes updates when data changes:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    RESOURCE SUBSCRIPTION FLOW                            │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   CLIENT                                              SERVER            │
│                                                                          │
│      │                                                   │              │
│      │  resources/subscribe                              │              │
│      │  {uri: "db://stock/AAPL"}                        │              │
│      │ ─────────────────────────────────────────────────►│              │
│      │                                                   │              │
│      │  OK (subscription confirmed)                      │              │
│      │ ◄─────────────────────────────────────────────────│              │
│      │                                                   │              │
│      │         ... time passes, data changes ...         │              │
│      │                                                   │              │
│      │  notifications/resources/updated                  │              │
│      │  {uri: "db://stock/AAPL"}                        │              │
│      │ ◄─────────────────────────────────────────────────│              │
│      │                                                   │              │
│      │  Client fetches new data via resources/read       │              │
│      │                                                   │              │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# resources/subscribe example
subscribe_request = {
    "jsonrpc": "2.0",
    "id": 20,
    "method": "resources/subscribe",
    "params": {
        "uri": "db://stock/AAPL"
    }
}

# Server notification when data changes (no request ID - it's a notification)
resource_updated_notification = {
    "jsonrpc": "2.0",
    "method": "notifications/resources/updated",
    "params": {
        "uri": "db://stock/AAPL"
    }
}

pprint(subscribe_request, "REQUEST: resources/subscribe")
pprint(resource_updated_notification, "NOTIFICATION: Resource Updated")

## 4. Prompts: Workflow Guidance

### 4.1 Understanding Prompts

**Prompts** are pre-defined templates that structure AI interactions:

| Aspect | Description |
|--------|-------------|
| Purpose | Guide model behavior, encode best practices |
| Arguments | Dynamic placeholders filled at runtime |
| Control | **User-controlled** - explicitly invoked |
| Use Case | Multi-step workflows, specialized tasks |

In [ ]:
# Example prompts
prompts = [
    {
        "name": "plan-vacation",
        "description": "Plan a complete vacation with flights, hotels, and activities",
        "arguments": [
            {
                "name": "destination",
                "description": "Destination city or region",
                "required": True
            },
            {
                "name": "startDate",
                "description": "Trip start date (YYYY-MM-DD)",
                "required": True
            },
            {
                "name": "endDate",
                "description": "Trip end date (YYYY-MM-DD)",
                "required": True
            },
            {
                "name": "budget",
                "description": "Total budget in USD",
                "required": False
            },
            {
                "name": "interests",
                "description": "List of interests (e.g., museums, beaches, food)",
                "required": False
            }
        ]
    },
    {
        "name": "code-review",
        "description": "Perform a thorough code review with security focus",
        "arguments": [
            {
                "name": "language",
                "description": "Programming language",
                "required": True
            },
            {
                "name": "focusAreas",
                "description": "Areas to focus on (security, performance, style)",
                "required": False
            }
        ]
    },
    {
        "name": "meeting-summary",
        "description": "Summarize meeting notes into actionable items",
        "arguments": [
            {
                "name": "meetingType",
                "description": "Type of meeting (standup, planning, review)",
                "required": True
            },
            {
                "name": "participants",
                "description": "List of participant names",
                "required": False
            }
        ]
    }
]

pprint({"prompts": prompts}, "AVAILABLE PROMPTS")

### 4.2 Prompt Methods

In [ ]:
# prompts/list - Discover available prompts
prompts_list_request = {
    "jsonrpc": "2.0",
    "id": 30,
    "method": "prompts/list",
    "params": {}
}

pprint(prompts_list_request, "REQUEST: prompts/list")

In [ ]:
# prompts/get - Retrieve a specific prompt with filled arguments
prompts_get_request = {
    "jsonrpc": "2.0",
    "id": 31,
    "method": "prompts/get",
    "params": {
        "name": "plan-vacation",
        "arguments": {
            "destination": "Barcelona, Spain",
            "startDate": "2025-06-15",
            "endDate": "2025-06-22",
            "budget": "5000",
            "interests": "architecture, food, beaches"
        }
    }
}

prompts_get_response = {
    "jsonrpc": "2.0",
    "id": 31,
    "result": {
        "description": "Plan a vacation to Barcelona, Spain",
        "messages": [
            {
                "role": "user",
                "content": {
                    "type": "text",
                    "text": """Plan a vacation with the following details:
                    
Destination: Barcelona, Spain
Dates: June 15-22, 2025 (7 days)
Budget: $5,000 total
Interests: architecture, food, beaches

Please:
1. Search for available flights
2. Find suitable hotels near attractions
3. Suggest daily itinerary based on interests
4. Recommend restaurants and local experiences
5. Provide cost breakdown staying within budget

Use the available tools to search for real options."""
                }
            }
        ]
    }
}

pprint(prompts_get_request, "REQUEST: prompts/get")
pprint(prompts_get_response, "RESPONSE: Filled Prompt")

### 4.3 Prompt Design Patterns

Effective prompts combine instructions with tool guidance:

In [ ]:
# Example: A well-designed prompt template
security_audit_prompt = {
    "name": "security-audit",
    "description": "Perform a comprehensive security audit on code",
    "arguments": [
        {"name": "codeUri", "description": "URI to the code resource", "required": True},
        {"name": "severity", "description": "Minimum severity level (low/medium/high/critical)", "required": False}
    ],
    "_template": """
You are a security expert performing a code audit.

## Instructions
1. First, use `resources/read` to fetch the code from: {codeUri}
2. Analyze for the following vulnerability categories:
   - SQL Injection
   - XSS (Cross-Site Scripting)
   - Authentication/Authorization flaws
   - Sensitive data exposure
   - Security misconfigurations
   - Insecure dependencies

3. For each finding, report:
   - Severity: {severity} or higher
   - Location (file, line number)
   - Description of the vulnerability
   - Recommended fix
   - CWE reference if applicable

4. Use the `checkDependencies` tool to scan for known CVEs
5. Use the `createIssue` tool to create tickets for critical findings

## Output Format
Provide a structured report with an executive summary followed by detailed findings.
"""
}

pprint(security_audit_prompt, "WELL-DESIGNED PROMPT: Security Audit")

## 5. Control Models: Who Decides What?

Understanding control is crucial for safe agentic workflows:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    MCP CONTROL HIERARCHY                                 │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   ┌─────────────────────────────────────────────────────────────────┐   │
│   │                     USER CONTROL                                 │   │
│   │                     ════════════                                 │   │
│   │   PROMPTS: User explicitly invokes workflows                     │   │
│   │   • Clicks "Plan Vacation" button                                │   │
│   │   • Types slash command                                          │   │
│   │   • Selects from menu                                            │   │
│   └─────────────────────────────────────────────────────────────────┘   │
│                              │                                          │
│                              ▼                                          │
│   ┌─────────────────────────────────────────────────────────────────┐   │
│   │                  APPLICATION CONTROL                             │   │
│   │                  ═══════════════════                             │   │
│   │   RESOURCES: App/User decides what context to fetch              │   │
│   │   • File browser for document selection                          │   │
│   │   • Search interface for knowledge base                          │   │
│   │   • Auto-fetch recent calendar events                            │   │
│   └─────────────────────────────────────────────────────────────────┘   │
│                              │                                          │
│                              ▼                                          │
│   ┌─────────────────────────────────────────────────────────────────┐   │
│   │                    MODEL CONTROL                                 │   │
│   │                    ═════════════                                 │   │
│   │   TOOLS: AI decides when to call tools                          │   │
│   │   • Model reasons about next action                              │   │
│   │   • Selects appropriate tool from available set                  │   │
│   │   • Constructs valid arguments                                   │   │
│   │                                                                  │   │
│   │   ⚠️ Should include user approval for sensitive actions!        │   │
│   └─────────────────────────────────────────────────────────────────┘   │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Control model summary
@dataclass
class ControlModel:
    primitive: str
    controller: str
    description: str
    safety_note: str

control_models = [
    ControlModel(
        primitive="Tools",
        controller="Model-controlled",
        description="AI decides when to call based on reasoning",
        safety_note="Implement user approval for destructive actions"
    ),
    ControlModel(
        primitive="Resources",
        controller="Application-controlled",
        description="App/User selects what context to provide",
        safety_note="Filter sensitive data before providing to model"
    ),
    ControlModel(
        primitive="Prompts",
        controller="User-controlled",
        description="User explicitly invokes workflow templates",
        safety_note="Validate prompt arguments before execution"
    )
]

print("\nMCP CONTROL MODELS")
print("=" * 70)
for cm in control_models:
    print(f"\n{cm.primitive}")
    print(f"  Controller: {cm.controller}")
    print(f"  Description: {cm.description}")
    print(f"  ⚠️ Safety: {cm.safety_note}")

## 6. Putting It All Together: A Complete Example

Let's see how Tools, Resources, and Prompts work together in a travel planning scenario:

In [ ]:
# Complete Travel MCP Server Definition
travel_server = {
    "serverInfo": {
        "name": "TravelPlannerMCP",
        "version": "1.0.0"
    },
    "capabilities": {
        "tools": {"listChanged": True},
        "resources": {"subscribe": True, "listChanged": True},
        "prompts": {"listChanged": True}
    },
    "tools": [
        {
            "name": "searchFlights",
            "description": "Search for available flights",
            "inputSchema": {
                "type": "object",
                "properties": {
                    "origin": {"type": "string"},
                    "destination": {"type": "string"},
                    "date": {"type": "string", "format": "date"}
                },
                "required": ["origin", "destination", "date"]
            }
        },
        {
            "name": "searchHotels",
            "description": "Search for available hotels",
            "inputSchema": {
                "type": "object",
                "properties": {
                    "city": {"type": "string"},
                    "checkIn": {"type": "string", "format": "date"},
                    "checkOut": {"type": "string", "format": "date"},
                    "guests": {"type": "integer"}
                },
                "required": ["city", "checkIn", "checkOut"]
            }
        },
        {
            "name": "bookFlight",
            "description": "Book a flight (requires user confirmation)",
            "inputSchema": {
                "type": "object",
                "properties": {
                    "flightId": {"type": "string"},
                    "passengers": {"type": "array", "items": {"type": "object"}}
                },
                "required": ["flightId", "passengers"]
            }
        },
        {
            "name": "createCalendarEvent",
            "description": "Create calendar event for trip",
            "inputSchema": {
                "type": "object",
                "properties": {
                    "title": {"type": "string"},
                    "startDate": {"type": "string", "format": "date-time"},
                    "endDate": {"type": "string", "format": "date-time"},
                    "description": {"type": "string"}
                },
                "required": ["title", "startDate", "endDate"]
            }
        }
    ],
    "resources": [
        {
            "uri": "travel://preferences/user",
            "name": "User Travel Preferences",
            "description": "User's saved travel preferences and loyalty programs",
            "mimeType": "application/json"
        },
        {
            "uri": "calendar://availability/next-month",
            "name": "Calendar Availability",
            "description": "User's available dates for next month",
            "mimeType": "application/json"
        }
    ],
    "resourceTemplates": [
        {
            "uriTemplate": "travel://past-trips/{destination}",
            "name": "Past Trip History",
            "description": "Previous trips to a destination"
        },
        {
            "uriTemplate": "weather://forecast/{city}/{date}",
            "name": "Weather Forecast",
            "description": "Weather forecast for planning"
        }
    ],
    "prompts": [
        {
            "name": "plan-vacation",
            "description": "Complete vacation planning workflow",
            "arguments": [
                {"name": "destination", "required": True},
                {"name": "dates", "required": True},
                {"name": "budget", "required": False}
            ]
        },
        {
            "name": "quick-weekend-trip",
            "description": "Plan a quick weekend getaway",
            "arguments": [
                {"name": "maxDistance", "required": False},
                {"name": "interests", "required": False}
            ]
        }
    ]
}

pprint(travel_server, "COMPLETE TRAVEL MCP SERVER")

### Workflow Execution Example

```
┌─────────────────────────────────────────────────────────────────────────┐
│            TRAVEL PLANNING WORKFLOW WITH MCP                             │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   1. USER invokes "plan-vacation" PROMPT                                │
│      └─► Arguments: {destination: "Barcelona", dates: "Jun 15-22"}      │
│                                                                          │
│   2. APP fetches RESOURCES for context                                  │
│      ├─► travel://preferences/user                                      │
│      ├─► calendar://availability/next-month                             │
│      └─► travel://past-trips/Spain                                      │
│                                                                          │
│   3. MODEL calls TOOLS based on reasoning                               │
│      ├─► searchFlights("JFK", "BCN", "2025-06-15")                      │
│      ├─► searchHotels("Barcelona", ...)                                 │
│      ├─► weather://forecast/Barcelona/2025-06-15  (resource)            │
│      └─► bookFlight("DL123", [...])  ⚠️ User approval required          │
│                                                                          │
│   4. Results compiled and presented to USER                             │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

## 7. Key Takeaways

### Tools
- ✅ Executable functions with JSON Schema validation
- ✅ Model-controlled (AI decides when to use)
- ✅ Should be single-purpose and well-documented
- ⚠️ Implement user approval for destructive actions

### Resources
- ✅ Read-only data sources identified by URIs
- ✅ Application/User-controlled (you choose what context)
- ✅ Support subscriptions for real-time updates
- ✅ Use templates for parameterized queries

### Prompts
- ✅ Pre-defined workflow templates
- ✅ User-controlled (explicitly invoked)
- ✅ Guide model behavior and encode best practices
- ✅ Combine instructions with tool/resource usage

### Next Steps

In **Notebook 3: Transport Mechanisms - STDIO**, we'll:
- Build a local MCP server using STDIO transport
- Understand subprocess communication
- Test with the MCP Inspector

## 8. Exercises

### Exercise 1: Design a Tool Schema

Create a JSON Schema for a `createGitHubIssue` tool that:
- Requires: repository owner, repository name, issue title
- Optional: issue body, labels (array), assignees (array)

In [ ]:
# Exercise 1: Complete this tool schema
github_issue_tool = {
    "name": "createGitHubIssue",
    "description": "___",  # Add description
    "inputSchema": {
        "type": "object",
        "properties": {
            # Add your properties here
        },
        "required": []  # Add required fields
    }
}

pprint(github_issue_tool, "YOUR TOOL SCHEMA")

### Exercise 2: Design Resource URIs

Design appropriate URIs for:
1. A direct resource for the current user's Slack status
2. A template for fetching a specific Slack channel's messages

In [ ]:
# Exercise 2: Design your resources
slack_resources = {
    "directResource": {
        "uri": "___",  # Design URI
        "name": "___",
        "description": "___"
    },
    "resourceTemplate": {
        "uriTemplate": "___",  # Design template URI
        "name": "___",
        "description": "___"
    }
}

pprint(slack_resources, "YOUR SLACK RESOURCES")

### Exercise 3: Create a Prompt Template

Design a prompt for "daily-standup-summary" that:
- Takes team name and date as arguments
- Instructs the AI to fetch calendar and Slack resources
- Uses tools to create a summary document

In [ ]:
# Exercise 3: Design your prompt
standup_prompt = {
    "name": "daily-standup-summary",
    "description": "___",
    "arguments": [
        # Add your arguments
    ]
}

pprint(standup_prompt, "YOUR STANDUP PROMPT")

---

## References

- [MCP Server Concepts](https://modelcontextprotocol.io/docs/learn/server-concepts)
- [JSON Schema Reference](https://json-schema.org/understanding-json-schema/)
- [MCP Primitives Specification](https://modelcontextprotocol.io/specification)

---

**Next Notebook:** [03_Transport_Mechanisms_STDIO.ipynb](./03_Transport_Mechanisms_STDIO.ipynb)